In [1]:
import pandas as pd
import os
import shutil

In [4]:
# create the data for postive samples (github)
FILE_PATH = 'metadata.csv'
IMAGE_PATH = 'images'


In [5]:
dataset = pd.read_csv(FILE_PATH)
print(dataset.shape)

(758, 29)


In [6]:
dataset['filename']

0      auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...
1      auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...
2      auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...
3      auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...
4                                  nejmc2001573_f1a.jpeg
                             ...                        
753                                         7a9ec606.jpg
754                                         88859dc1.jpg
755                                         3a81faf3.jpg
756                                         bcb814aa.jpg
757                                         76093afc.jpg
Name: filename, Length: 758, dtype: object

In [7]:
TARGET_DIR = 'dataset/Covid'
if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)

In [8]:
cnt = 0
for (i, row) in dataset.iterrows():
    if row['finding'] == 'COVID-19' and row['view'] == 'PA':
        filename = row['filename']
        image_path = IMAGE_PATH + '/' + filename
        image_copy_path = TARGET_DIR + '/' + filename
        shutil.copyfile(image_path,image_copy_path)
        cnt += 1

# print(cnt)

In [9]:
# Sampling of images from kaggle
import random
KAGGLE_FILE_PATH = 'test/NORMAL'
TARGET_NORMAL_DIR = 'dataset/Normal'


In [10]:
image_names = os.listdir(KAGGLE_FILE_PATH)
# image_names

In [11]:
random.shuffle(image_names)

In [12]:
for i in range(142):
    image_name = image_names[i]
    image_path = KAGGLE_FILE_PATH + '/' + image_name
    target_path = TARGET_NORMAL_DIR + '/' + image_name
    shutil.copyfile(image_path, target_path)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/Normal/NORMAL2-IM-0368-0001.jpeg'

In [2]:
# Building CNN
TRAIN_PATH = 'dataset/Train'
VAL_PATH = 'dataset/Validation'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

Using TensorFlow backend.


In [4]:
model = Sequential()

# adding input layer
model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu', input_shape = [224, 224, 3]))
# cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [64, 64, 3]))
# model.add(MaxPooling2D(pool_size =2, strides = 2))

# adding secong layer
# filters 64 used for increse the non linearity
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size =2, strides = 2))
model.add(Dropout(0.25))

# add another layer
model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size =2, strides = 2))
model.add(Dropout(0.25))


# flatterning
model.add(Flatten())

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

# outputlayer
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss= keras.losses.binary_crossentropy, optimizer = 'adam', metrics = ['accuracy'])

In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 54, 54, 128)      

In [6]:
# train from scratch
train_datagen = image.ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_dataset = image.ImageDataGenerator(rescale = 1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(
'dataset/Train',
    target_size = [224,224],
    batch_size = 32,
    class_mode = 'binary'
)

Found 343 images belonging to 2 classes.


In [8]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [9]:
validation_generator = test_dataset.flow_from_directory(
'dataset/Validation',
    target_size = [224,224],
    batch_size = 32,
    class_mode = 'binary'
)

Found 60 images belonging to 2 classes.


In [10]:
model.fit_generator(
train_generator,
steps_per_epoch = 8,
epochs =10,
validation_data = validation_generator,
validation_steps = 2)

Epoch 1/10
8/8 [==============================] - 66s 8s/step - loss: 5.2116 - accuracy: 0.4980 - val_loss: 0.7053 - val_accuracy: 0.5000
Epoch 2/10
8/8 [==============================] - 53s 7s/step - loss: 0.6832 - accuracy: 0.5992 - val_loss: 0.7242 - val_accuracy: 0.2167
Epoch 3/10
8/8 [==============================] - 51s 6s/step - loss: 0.5712 - accuracy: 0.7368 - val_loss: 0.8188 - val_accuracy: 0.2167
Epoch 4/10
8/8 [==============================] - 49s 6s/step - loss: 0.4432 - accuracy: 0.8203 - val_loss: 0.9381 - val_accuracy: 0.2167
Epoch 5/10
8/8 [==============================] - 49s 6s/step - loss: 0.3935 - accuracy: 0.8178 - val_loss: 1.0362 - val_accuracy: 0.3333
Epoch 6/10
8/8 [==============================] - 52s 6s/step - loss: 0.3766 - accuracy: 0.8281 - val_loss: 1.0246 - val_accuracy: 0.3667
Epoch 7/10
8/8 [==============================] - 52s 7s/step - loss: 0.3476 - accuracy: 0.8138 - val_loss: 1.0897 - val_accuracy: 0.3167
Epoch 8/10
8/8 [==================

In [37]:
# single Prediction
normal_image_path = 'dataset/val/NORMAL'
covid_image_path = 'dataset/val/PNEUMONIA'
imagesdirlist = [normal_image_path, covid_image_path]
from os import listdir


for imagedir in imagesdirlist:
    for file in listdir(imagedir):
        
        if file.startswith('NORMAL'):
            path = normal_image_path + '/' + file
        elif  file.startswith('person'):
            path = covid_image_path + '/' + file
            
        test_image = image.load_img(path, target_size = (224, 224))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = model.predict(test_image)
        if result[0][0] == 1:
            print('prediction is right')
        else:
            print('prediction is wrong')
        if file.startswith('NORMAL'):
            if result[0][0] == 1:
                print('prediction is right')
            else:
                print('prediction is wrong')
        elif file.startwith('person'):
            if result[0][0] == 1:
                print('prediction is wrong')
            else:
                print('prediction is right')
            

prediction is right
prediction is right
prediction is right
prediction is right
prediction is right
prediction is right
prediction is right
prediction is right
prediction is right
prediction is right
prediction is right
prediction is right
prediction is right
prediction is right
prediction is right
prediction is right
prediction is right


AttributeError: 'str' object has no attribute 'startwith'